[Retour au sommaire](../index.ipynb)

# TP Stéganographie

La stéganographie est l'art de la dissimulation.

Dans ce TP nous allons dissimuler une image dans une autre.

- L'image qui reçoit est appelée **l'image hôte**.
- L'image qui doit se dissumuler est appelée **l'image invitée**.

Nous pourrions cacher :

- du texte dans une image
- une image dans un son
- un son dans une image...

le principe resterait le même.

## Principe

Toutes les images matricielles contiennent des pixels.

Chaque pixel d'une image est composée de 3 composantes : Rouge, Vert et Bleu. (RVB)

Si on prend une image codée sur 24bits (3 octets), chaque composante de la couleur est donc codée sur 1 octet.

Il y a donc :

- 256 nuances de rouge possibles (de 0 à 255)
- 256 nuances de vert possibles (de 0 à 255)
- 256 nuances de bleu possibles (de 0 à 255)

Dans ce type d'image sur 24 bits, il y a donc $2^{24} \approx 16 700 000$ couleurs possibles.


Considérons, par exemple, la **composante rouge** d'**un pixel** de l'image:

Si on modifie, les **deux bits de poids faible**, au maximum, on aura une différence de 3 avec la composante rouge d'origine : 

 - si j'ai 00 -> 11 (3 de difference maximale)
 - si j'ai 01 -> 11 (2 de différence maximale)
 - si j'ai 10 -> 00 (2 de difference maximale)
 - si j'ai 11 -> 00 (3 de difference maximale)
 

On obtient donc, **dans le pire des cas**, une différence de 3 par rapport à la valeur initiale. Et ce sur 256 nuances possibles.
 
 **La différence est si minime qu'il est impossible de la voir à l'oeil**
 
 Voici par exemple une image **uniquement à base de rouge**  (V=0, B=0):
 
 <table width="25%" border="1px solid black">
    <tr><td>La partie gauche contient une valeur de rouge qui est à 192 : 11000000</td><td>La partie droite contient une valeur de rouge qui est à 195 : 11000011</td></tr>
 </table>
 
 ![difference de 3 sur du rouge](img/diff_couleur.png)
 
Faisons le test sur les 3 derniers bits.

 <table width="25%" border="1px solid black">
    <tr><td>La partie gauche contient une valeur de rouge qui est à 192 : 11000000</td><td>La partie droite contient une valeur de rouge qui est à 199 : 11000111</td></tr>
 </table>
 
 ![difference de 3 sur du rouge](img/diff_couleur2.png)
 
 Là on commence à distinguer une différence de couleur.
 
 
<div class="alert alert-info">Nous allons nous contenter de modifier <b>les deux derniers bits</b> des composantes R, V et B de chaque pixel de notre image hôte afin exploiter cette 'réserve' de bits pour <b>cacher</b> notre seconde image.</div>

## Premiere approche

### Phase de dissimulation

On utilise une image **hôte** et une image **invitée** de **dimensions égales**.

Les bits de poids forts (MSB : **M**ost **S**ignificant **B**it) sont situés à gauche (128/0 pour le premier, 64/0 pour le second)

Les bits de poids faibles (LSB : **L**ess **S**ignificant **B**it) sont situés à droite (1/0 pour le premier, 2/0 pour le second)

<div class="alert alert-info">Ainsi, si je conserve uniquement les deux bits de poids fort de chacune des 3 composantes de chaque pixel de l'image invitée, je conserverai <b>grossièrement</b> les couleurs d'origine .

Si je "cache" ces deux bits de poids fort dans les deux bits de poids faible de mon image hôte, je ne changerai pratiquement pas la valeur de la composante de l'hôte.</div>


La stratégie est donc la suivante : Pour chaque pixel des images, et pour chaque composante de couleur :

- Je récupère les deux bits de poids fort de l'image invité
- Je remplace les deux bits de poids faible de l'hôte par les valeurs des 2 bits de poids fort de l'invité.

![schéma](img/stegano_schema1.png)

**Remarque**

On peut donc stocker uniquement la valeur des 3 composantes de l'image à cacher sur 6 bits soit: 


**Ce qui donne $2^6=64$ couleurs possibles pour notre image dissimulée.**

Lors de la phase d'extraction, l'image qui se révèlera sera donc dégradée en terme de nombres de couleurs.

### Phase de d'extraction

- On crée une image blanche de la même dimension
- On parcourt tous les pixels de l'image qui contient notre image dissimulée.
  - On recupère les deux dernièrs bits de chaque composante RVB
  - Puis on ajoute 6 bits derrière chaque composante. A tester faut il ajouter :
    - 000000
    - 111111
    - 101010 ?
  - on injecte ces valeurs dans le pixel de notre image blanche.
- on retourne notre image blanche (qui ne l'est plus...)



### Echauffement...

#### Comprendre comment extraire les valeurs de toutes les composantes (R, V, B) de tous les pixels de l'image

Voici une image de deux pixels sur trois pixels grossie :

<img src="img/image_3_2_grossie.png">

Nous allons utiliser la librairie [PIL](https://pillow.readthedocs.io/en/stable/) ( **P**ython **I**mage **L**ibrairie ) afin de parcourir les pixels de cette image.

In [50]:
from PIL import Image # importation du module Image de la librairie PIL

image = Image.open("img/image_3_2.bmp") # je précise le chemin de mon image
largeur, hauteur = image.size # image.size retourne un tuple de 2 valeurs

print(f"largeur = {largeur} pixels")
print(f"hauteur = {hauteur} pixels")

#pixels = image.load() # j'ai tous les pixels de l'image grace à load()

#for y in range(hauteur): # je boucle sur la hauteur
#    print("------------------------------> NOUVELLE LIGNE")
#    for x in range(largeur): # je boucle sur la largeur
#        print(f"\nJe suis sur le pixel de coordonnées ({x},{y})")
        ### On récupère les valeurs r, v, b d'un pixel en donnant les coord (x, y) du pixel
        ### pixels est un peu comme un tableau de tableau
#        r, v, b = pixels[x, y] 
#        print(f"composante rouge = {r} = {bin(r)}")
#        print(f"composante verte = {v} = {bin(v)}")
#        print(f"composante bleue = {b} = {bin(b)}")


largeur = 3 pixels
hauteur = 2 pixels


#### Comprendre comment fusionner deux composantes ( l'hôte et l'invité)

Imaginons que 

- la composante rouge du pixel de coordonnées (x, y) de l'**hôte** est 224
- la composante rouge du pixel de coordonnées (x, y) de l'**invité** est 190

In [54]:
r_hote = 224
r_invite = 190
### Je passe en binaire
r_hote_bin = bin(r_hote)
r_invite_bin = bin(r_invite)
print(f"La valeur de l'hôte en binaire est {r_hote_bin}")
print(f"La valeur de l'invité en binaire est {r_invite_bin}")

### bin(nombre) renvoie une string, j'enleve le '0b'
#r_hote_bin = r_hote_bin[2:]
#r_invite_bin = r_invite_bin[2:]
#print(f"La valeur de l'hôte est {r_hote_bin}.")
#print(f"La valeur de l'invité est {r_invite_bin}.")

### J'enleve les deux derniers bits de l'hote (LSB)
#r_hote_bin = r_hote_bin[:-2]
#print(f"La valeur tronquée de l'hôte est maintenant de {r_hote_bin}.")
### Je garde les deux premiers bits de l'invité (MSB)
#r_invite_bin = r_invite_bin[0:2]
#print(f"les deux premiers bits de l'invité sont {r_invite_bin}.")

#fusion = r_hote_bin + r_invite_bin
#print(f"La fusion donne {fusion}.")

### Ne reste plus qu'à repasser cela en nombre.

#valeur_fusion = int(fusion, 2) # le 2 est pour en binaire
#print(f"Le nombre fusionné est maintenant {valeur_fusion}.")



La valeur de l'hôte en binaire est 0b11100000
La valeur de l'invité en binaire est 0b10111110
La valeur de l'hôte est 11100000.
La valeur de l'invité est 10111110.
La valeur tronquée de l'hôte est maintenant de 111000.
les deux premiers bits de l'invité sont 10.
La fusion donne 11100010.
Le nombre fusionné est maintenant 226.


#### Tout se passait si bien...

Prenons maintenant : 

- r_hote = 224
- r_invite = 6




In [55]:
r_hote = 224
r_invite = 6
### Je passe en binaire
r_hote_bin = bin(r_hote)
r_invite_bin = bin(r_invite)
print(f"La valeur de l'hôte est {r_hote_bin}")
print(f"La valeur de l'invité est {r_invite_bin}")

print("Aïe Aïe Aïe... : regardez la valeur de l'invité !!! Vous avez compris le problème ?")
print("+------------------------------------------------+")
print("|  J'espère car c'est à vous de le résoudre :-)  |")
print("+------------------------------------------------+")


La valeur de l'hôte est 0b11100000
La valeur de l'invité est 0b110
Aïe Aïe Aïe... : regardez la valeur de l'invité !!! Vous avez compris le problème ?
+------------------------------------------------+
|  J'espère car c'est à vous de le résoudre :-)  |
+------------------------------------------------+


## Deuxième approche (projet ?)

Plutôt que de diminuer le nombre de couleurs, nous allons diminuer la définition de notre image invitée.

Si on divise par 2, la largeur et la hauteur, le nombre de pixels est divisé par 4.

Voici un schéma pour la composante rouge d'un pixel: 

![schéma](img/stegano_schema.png)

Les 8 bits de la composante Rouge sont dissimulés par paquet de 2 dans la composante rouge de 4 pixels de l'hôte.

**Notre image invitée conservera la qualité de ses couleurs mais sera 4 fois plus petite.**



[Retour au sommaire](../index.ipynb)